In [1]:
import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) <= 3)

!ls /kaggle/working/

MODEL_WEIGHTS_SRC ="/kaggle/input/agemagician-mlong-t5-tglobal-large-model/agemagician_mlong-t5-tglobal-large/agemagician_mlong-t5-tglobal-large"

MODEL_NAME = "agemagician_mlong-t5-tglobal-large"
# MODEL_NAME = "UBC-NLP/AraT5v2-base-1024" #"agemagician_mlong-t5-tglobal-large" #"facebook_mbart-large-50-many-to-many-mmt" # "UBC-NLP_AraT5-base" #"google/mt5-small"

MODEL_WEIGHTS_DEST_DIR =f"/kaggle/working/models/fine_tuned_model/model_weights/{MODEL_NAME}"



# !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    !mkdir -p {MODEL_WEIGHTS_DEST_DIR}
    !cp -r {MODEL_WEIGHTS_SRC} {MODEL_WEIGHTS_DEST_DIR}
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")



# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

analysis   data        __init__.py  prompts	 run_pipeline.py  utils
config.py  evaluation  models	    __pycache__  state.db
❌ '/kaggle/working' is not empty — contains 12 items.
/kaggle/working
analysis   data        __init__.py  prompts	 run_pipeline.py  utils
config.py  evaluation  models	    __pycache__  state.db


In [2]:
!ls -R {MODEL_WEIGHTS_DEST_DIR}

/kaggle/working/models/fine_tuned_model/model_weights/facebook_mbart-large-50-many-to-many-mmt/:
config.json		sentencepiece.bpe.model  training_args.bin
generation_config.json	special_tokens_map.json
model.safetensors	tokenizer_config.json


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# project_dir = "/content/drive/MyDrive/AraGenEval2025"
# os.chdir(project_dir)
# print(f"✅ Current directory: {os.getcwd()}")


In [5]:
!pip install transformers[sentencepiece] torch pandas openpyxl psutil python-dotenv  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.8 MB/s eta 0:00:00:00:0100:01


In [7]:
# Ensure PyTorch can expand allocations if needed
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [8]:

import os
import json
import dotenv
import torch
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import gc
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from config import Config


# project root in PYTHONPATH
import sys
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"✅ Using device: {device}")

# Device & parallel setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}, count: {torch.cuda.device_count()} GPUs")

✅ Using device: cuda, count: 2 GPUs


In [9]:

def safe_model_name(model_name):
    return model_name.replace("/", "_")

# archive & inspect
def create_run_dir(base, name):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    d  = os.path.join(base, safe_model_name(name), f"run_{ts}")
    os.makedirs(d, exist_ok=True)
    return d

def run_evaluation():
  pass
    # os.system("python utils/prepare_metrics_log.py")
    # os.system("python evaluation/evaluate.py")


def archive_results(run_dir, files_to_archive):
    for fpath in files_to_archive:
        if os.path.exists(fpath):
            dest = os.path.join(run_dir, os.path.basename(fpath))
            os.system(f"cp {fpath} {dest}")


In [10]:
def load_model(model_name: str):
    secure = safe_model_name(model_name)
    model_path = os.path.join("models/fine_tuned_model/model_weights", secure)
    print(f"Loading model from {model_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, legacy=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    # mBART language tags
    if "mbart" in model_name.lower():
        tokenizer.src_lang = tokenizer.tgt_lang = "ar_AR"
    # wrap for multi‑GPU
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    model.to(device).eval()
    return tokenizer, model  


In [11]:
### batched inference function (updated for DataParallel)
def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
    # 1. Load & prepare
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)

    # Handle DataParallel wrapping
    base_model = model.module if hasattr(model, "module") else model
    base_model.generation_config.pad_token_id = tokenizer.pad_token_id
    assert base_model.config.eos_token_id is not None

    records = []
    # 2. DataLoader for batching
    from torch.utils.data import DataLoader, Dataset

    class TextDataset(Dataset):
        def __init__(self, texts): self.texts = texts
        def __len__(self): return len(self.texts)
        def __getitem__(self, idx): return self.texts[idx]

    def collate_fn(batch_texts):
        return tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,    # preserve full inputs
            max_length=1300
        )

    loader = DataLoader(
        TextDataset(df["input_text"].tolist()),
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=4,
        pin_memory=True
    )

    # 3. Generate in batches
    for i, batch in enumerate(tqdm(loader, desc="Batches")):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.inference_mode():
            gen_ids = base_model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                num_beams=2,
                early_stopping=True,
                do_sample=False,
                max_length=3000
                max_new_tokens=1700
            )
        outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

        # accumulate
        start_idx = i * batch_size
        for j, out in enumerate(outs):
            row = df.iloc[start_idx + j]
            records.append({
                "id": int(row["id"]),
                "author": row["author"],
                "output": out,
                "ground_truth": row["text_in_author_style"]
            })

        torch.cuda.empty_cache()
        gc.collect()

    # 4. Save
    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)
    print(f"✅ Generated {len(records)} records to {output_log_path}")

In [12]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

41

In [13]:
!nvidia-smi

Mon Jul 14 23:37:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
# Cell 4: run the pipeline
# adjust these paths as needed
MODEL_NAME      = "agemagician_mlong-t5-tglobal-large"
validation_file = Config.VAL_FILE
log_file        = f"models/fine_tuned_model/results/{safe_model_name(MODEL_NAME)}/generation_log.json"
output_base_dir = "analysis/fine_tuned_model"

os.makedirs(os.path.dirname(log_file), exist_ok=True)

tokenizer, model = load_model(MODEL_NAME)
generate_predictions(tokenizer, model, validation_file, log_file, batch_size=12)



run_dir = create_run_dir(output_base_dir, MODEL_NAME)
os.system(f"cp {log_file} {run_dir}/")
print(f"\n✅ Pipeline done. Logs in: {run_dir} \n LOG FILE: {log_file}\n")
# !nvidia-smi

Loading model from models/fine_tuned_model/model_weights/facebook_mbart-large-50-many-to-many-mmt


2025-07-14 23:37:22.573478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752536242.907644      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752536243.008725      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Batches: 100%|██████████| 520/520 [6:52:56<00:00, 47.65s/it]  


✅ Generated 4157 records to models/fine_tuned_model/results/facebook_mbart-large-50-many-to-many-mmt/generation_log.json

✅ Pipeline done. Logs in: analysis/fine_tuned_model/facebook_mbart-large-50-many-to-many-mmt/run_20250715_063037 
 LOG FILE: models/fine_tuned_model/results/facebook_mbart-large-50-many-to-many-mmt/generation_log.json

